In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#Converting jasonl files to csv
import csv
import json
import time

testing_csv = "testing_text.csv"
training_csv = "training_text.csv"
validating_csv = "validating_text.csv"

testing = "testing_summary.jsonl"
training = "training_summary.jsonl"
validating = "validation_summary.jsonl"

def writeTocsv(data):
    with open(validating_csv, 'a+', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(data)

with open(validating, 'r') as json_file:
    result = [dict(json.loads(jline)) for jline in json_file.read().splitlines()]

document = {}
keys_w = ['docid', 'title', 'authors', 'abstract', 'summary']
for i in range(len(result)):
  temp = {}
  for key in dict(result[i]):
    if key in keys_w:
      temp[key] = result[i][key]
  document[result[i]["docid"]] = temp
  # document[result[i]["docid"]] = dict(result[i])
print(type(document[result[0]["docid"]]))
print("Number of documents for testing:",len(document))
print("Keys in the document dictionary:",document[result[0]['docid']].keys())


'''def parse_general_references(general_references):
    title = []
    for dict in general_references:
      title.append(dict['title'])
    return title
for value in document.values():
    document[value['docid']]['general_references'] = parse_general_references(value["general_references"])
    '''
#print(header)
print("Starting to write to File")
writeTocsv(keys_wgaf)
for value in document.values():
    writeTocsv(value.values())
print("Completed!!")

def printJson(data):
    print(json.dumps(data,indent=2))

In [ ]:
#Removing PICO tags
import pandas as pd
import csv
import re

df = pd.read_csv("training.csv")

abs = df['abstract'].values
print('abstract', abs)


start = ['<pop>', '<int>', '<out>']
end = ['</pop>', '</int>', '</out>']
keys = ['pop', 'int', 'out']


def writeTocsv(file, data):
    with open(file, 'a+', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(data)


# The following function removes all PICO(HTML) tags from the data and keeps only the required string
def remove_PICO(abs_str):
    for k in range(len(keys)):
        summary[keys[k]] = re.findall(start[k]+"(.*?)"+end[k], abs_str, re.DOTALL)

    
    n = len(abs_str)
    sent = abs_str
    
    for i in start:
        if i in abs_str:
            sent = sent.replace(i, '')

    for i in end:
        if abs_str:
            sent = sent.replace(i, '')
        
    return sent

In [ ]:
#Read trainning data
train_summary = pd.read_csv('../input/meddoc/training_summary.csv', encoding='iso-8859-1')
train_text = pd.read_csv('../input/meddoc/training.csv', encoding='iso-8859-1')

In [4]:
#Read validation data
val_summary = pd.read_csv('../input/meddoc/val_summary.csv', encoding='iso-8859-1')
val_text = pd.read_csv('../input/meddoc/val.csv', encoding='iso-8859-1')

In [ ]:
#Read test data
test_summary = pd.read_csv('../input/meddoc/test_summary.csv', encoding='iso-8859-1')
test_text = pd.read_csv('../input/meddoc/test.csv', encoding='iso-8859-1')

In [ ]:
#Concating text and summary for train,test and validation
train_data=pd.concat([train_text,train_summary],ignore_index=True,axis=1)
val_data=pd.concat([val_text,val_summary],ignore_index=True,axis=1)
test_data=pd.concat([test_text,test_summary],ignore_index=True,axis=1)

In [5]:
train_data.head()

In [6]:
#Dropping empty columns in dataset
train_data=train_data.drop(columns=[2,4])
val_data=val_data.drop(columns=[2,4])
test_data=test_data.drop(columns=[2,4])

In [7]:
#Renaming the columns in dataset
train_data=train_data.rename(columns={0: "doc_id", 1: "title",3:"text",5:"summary"})
train_data.head()

In [8]:
val_data=val_data.rename(columns={0: "doc_id", 1: "title",3:"text",5:"summary"})
test_data=test_data.rename(columns={0: "doc_id", 1: "title",3:"text",5:"summary"})

In [9]:
#Cleaning text for tags, and puntuation and quotiatons
import re           
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords   
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString) 
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

#cleaning texts of train, test and  validation calling the above function
cleaned_train_text = []
cleaned_val_data=[]
cleaned_test_data=[]
for t in train_data['text']:
    cleaned_train_text.append(text_cleaner(t))
for v in val_data['text']:
    cleaned_val_text.append(text_cleaner(t))
for t in test_data['text']:
    cleaned_test_text.append(text_cleaner(t))

In [ ]:
#Cleaning summary for tags, and puntuation and quotiatons
def summary_cleaner(text):
    newString = re.sub('"','', text)    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString

#cleaning summary of train, test and  validation calling the above function
cleaned_train_summary = []
cleaned_val_summary=[]
cleaned_test_summary=[]
for t in train_data['summary']:
    cleaned_train_summary.append(summary_cleaner(t))
for v in val_data['summary']:
    cleaned_val_summary.append(summary_cleaner(t))
for t in test_data['summary']:
    cleaned_test_summary.append(summary_cleaner(t))


train_data['cleaned_train_text']=cleaned_train_text
train_data['cleaned_train_summary']=cleaned_train_summary
val_data['cleaned_val_text']=cleaned_val_text
val_data['cleaned_val_summary']=cleaned_val_summary
test_data['cleaned_test_text']=cleaned_test_text
test_data['cleaned_test_summary']=cleaned_test_summary
train_data['cleaned_train_summary'].replace('', np.nan, inplace=True)
val_data['cleaned_val_summary'].replace('', np.nan, inplace=True)
test_data['cleaned_test_summary'].replace('', np.nan, inplace=True)
train_data.dropna(axis=0,inplace=True)
val_data.dropna(axis=0,inplace=True)
test_data.dropna(axis=0,inplace=True)

In [ ]:
# plotting wordcloud 
def wc_blue_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(214, 67%%, %d%%)" % randint(60, 100)

def plot_wordcloud(text, color_func):
    wc_stopwords = get_wc_stopwords()
    wc = WordCloud(stopwords=wc_stopwords, width=1200, height=600, random_state=0).generate(text)

    f, axs = plt.subplots(figsize=(20, 10))
    with sns.axes_style("ticks"):
        sns.despine(offset=10, trim=True)
        plt.imshow(wc.recolor(color_func=color_func, random_state=0), interpolation="bilinear")
        plt.xlabel('WordCloud')

In [ ]:
plot_wordcloud(' '.join(train_data['cleaned_summary_data'].values.tolist()), wc_blue_color_func)

In [ ]:
#Adding start and end for each summary in train, test and validation to identify start and end of the texts
train_data['cleaned_train_summary'] = train_data['cleaned_train_summary'].apply(lambda x : '_start_ '+ x + ' _end_')
val_data['cleaned_val_summary'] = train_data['cleaned_val_summary'].apply(lambda x : '_start_ '+ x + ' _end_')
test_data['cleaned_test_summary'] = train_data['cleaned_test_summary'].apply(lambda x : '_start_ '+ x + ' _end_')

In [ ]:
for i in range(5):
    print("Review:",train_data['cleaned_train_text'][i])
    print("Summary:",train_data['cleaned_train_summary'][i])
    print("\n")

In [ ]:
ttext_count = []
tsummary_count = []

In [ ]:
for sent in train_data['cleaned_train_text']:
    ttext_count.append(len(sent.split()))
for sent in train_data['cleaned_train_summary']:
    tsummary_count.append(len(sent.split()))

In [ ]:
t_wordcount_df= pd.DataFrame()
t_wordcount_df['train_text']=ttext_count
t_wordcount_df['train_summary']=tsummary_count

In [ ]:
#Plot histogram to check range of words in text and summary in train data
import matplotlib.pyplot as plt

twordcount_df.hist(bins = 5)
plt.show()

In [ ]:
#To count number of words in train summary
cnt=0
for i in train_data['cleaned_train_summary']:
    if(len(i.split())<=300):
        cnt=cnt+1
print(cnt/len(train_data['cleaned_train_summary']))

In [ ]:
#To count number words in train text
cnt=0
for i in train_data['cleaned_train_text']:
    if(len(i.split())<=2000):
        cnt=cnt+1
print(cnt/len(train_data['cleaned_train_text']))

In [ ]:
#Setting text and summary words limit
max_text_len=6000
max_summary_len=60

In [ ]:
#Select the Summaries and Text between max len defined above

cleaned_train_text =np.array(train_data['cleaned_train_text'])
cleaned_train_summary=np.array(train_data['cleaned_train_summary'])
cleaned_val_text =np.array(train_data['cleaned_val_text'])
cleaned_val_summary=np.array(train_data['cleaned_val_summary'])
cleaned_test_text =np.array(train_data['cleaned_test_text'])
cleaned_test_summary=np.array(train_data['cleaned_test_summary'])

def shorten(cleaned_data,max_text_len,max_summary_len):
train_text_small=[]
train_summary_small=[]

for i in range(len(cleaned_data)):
    if(len(cleaned_train_summary[i].split())<=max_summary_len and len(cleaned_train_text[i].split())<=max_text_len):
        train_text_small.append(cleaned_train_text[i])
        train_summary_small.append(cleaned_train_summary[i])
return train_text_small,train_summary_small
               
train_text_small,train_summary_small=shorten(cleaned_train_text)        
st_data=pd.DataFrame({'text':train_text_small,'summary':train_summary_small})
val_text_small,val_summary_small=shorten(cleaned_val_text)        
sv_data=pd.DataFrame({'text':val_text_small,'summary':val_summary_small})
test_text_small,test_summary_small=shorten(cleaned_test_text)        
stt_data=pd.DataFrame({'text':test_text_small,'summary':test_summary_small})

In [ ]:
#Adding sostok and eostok at the end of summary
st_data['summary'] = st_data['summary'].apply(lambda x : 'sostok '+ x + ' eostok')
sv_data['summary'] = sv_data['summary'].apply(lambda x : 'sostok '+ x + ' eostok')
stt_data['summary'] = stt_data['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [ ]:
x_train=st_data['text']
y_train=st_data['summary']
x_val=sv_data['text']
y_val=sv_data['summary']
x_test=stt_data['text']
y_test=stt_data['summary']

In [ ]:
#Tokenizing to get vocabulary

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_train))

In [ ]:
threshold=4
count=0
total=0
frequent=0
total_f=0

for key,value in x_tokenizer.word_counts.items():
    total_f=total_f+value
    total=total+1
    if(threshold>value):
        count=count+1
        frequent=frequent+value
    
print("% of rare words in vocabulary:",(count/total)*100)
print("Total Coverage of rare words:",(frequent/total_f)*100)

In [ ]:
#Tokening training data text
x_tokenizer = Tokenizer(num_words=total-count) 
x_tokenizer.fit_on_texts(list(x_train))


#one-hot encodeing text to word
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_train) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)
#padding zero 
x_train    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')



#Vocabulary+1 for padding
x_vocab   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_vocab))

In [ ]:
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_train))

In [ ]:
threshold=6
count=0
total=0
frequent=0
total_f=0

for key,value in y_tokenizer.word_counts.items():
    total_f=total_f+value
    total=total+1
    if(threshold>value):
        count=count+1
        frequent=frequent+value
    
print("% of rare words in vocabulary:",(count/total)*100)
print("Total Coverage of rare words:",(frequent/total_f)*100)

In [ ]:
#Tokening training data summary
y_tokenizer = Tokenizer(num_words=total-count) 
y_tokenizer.fit_on_texts(list(y_train))

#one-hot encodeing text to word
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_train) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero 
y_train    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#Vocabulary+1 for padding
y_vocab  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_vocab))

In [ ]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

print("Size of vocabulary from the w2v model = {}".format(x_voc))

K.clear_session()

latent_dim = 300
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))


#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Model definition
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

In [ ]:
#Compiling the model
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy')

In [ ]:
#Fitting the model
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=20,batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))
model.save('model_1.h5', save_format = 'h5')

In [ ]:
Plotting the graph for train and validation loss
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
#Rexersing and getting back the string form of the summary
reverse_target_word_index=y_tokenizer.index_word 
reverse_source_word_index=y_tokenizer.index_word 
target_word_index=y_tokenizer.word_index

In [ ]:
# Encoder Inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder Inference

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_body,latent_dim))

# Getting decoder sequence embeddings
dec_emb2= dec_emb_layer(decoder_inputs)

# Predicting the next word in the sequence



decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# Dense softmax layer to calculate probability distribution over target vocab
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encoding input as state vectors
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Initializing target to 1
    target_seq = np.zeros((1,1))

    # Taking the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        try:
            sampled_token = reverse_target_word_index[sampled_token_index]
        except:
            sampled_token = reverse_target_word_index[np.random.randint(1, len(reverse_target_word_index))]
        if(sampled_token!='end'):
            decoded_sentence += ' '+sampled_token

            # Stop when max length is reached or find end word.
            if (sampled_token == 'end' or len(decoded_sentence.split()) >= (max_len_highlight-1)):
                stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
#Convet texts back to strings(decode)
def seq2text(input_sequence):
    new_string = ''
    for i in input_sequence:
        if i != 0:
            new_string = new_string + reverse_source_word_index[i] + ' '
    return new_string

In [ ]:
#Convert summary back to string(decode)
def seq2summary(input_sequence):
    new_string = ''
    for i in input_sequence:
        if (
            (i != 0 and i != target_word_index[start_token]) and
            (i != target_word_index[end_token])
        ):
            new_string = new_string + reverse_target_word_index[i] + ' '
    return new_string

In [ ]:
#Storing original and predicted to lists
original_summary=[]
predicted_summary=[]
for i in range(len(x_test)):
  original_summary.append(seq2text(x_test[i]))
  predicted summary.append(decode_sequence(x_test[i].reshape(1,max_summary_len)))


In [ ]:
#Writing original a predicted summary to a csv 
import csv

with open('./predict.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(original_summary,predicted_summary))

In [ ]:
!pip install rouge
import json
from rouge import Rouge
import pandas as pd
import numpy as np

#Reading the csv and writing original and predicted to lists
data=pd.read_csv('../input/sgd-evaluation/some1 (4).csv')
original=data['original_summary'].tolist()
predicted=data['predicted_summary'].tolist()

#Calculating the Rouge scores
rouge = Rouge()
scores = rouge.get_scores(original,predicted, avg=True)

print(scores)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
actual=[]
predicted=[]
#Spliting the words in each summary to perform bleu evaluataion
for i in range(len(original)):
    actual.append([original[i].split()])
    predicted.append(predicted[i].split())

#Finding bleu for 1,2,3 and 4 shingles
bleu = {}
bleu['1-grams'] = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
bleu['2-grams'] = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
bleu['3-grams'] = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
bleu['4-grams'] = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))

print(bleu)